# Problem 1: The time dependence of rock advection (*11.5 points*)

## Time-dependent temperatures in the Earth

In this exercise we will use an analytical solution to the 1-D time-dependent thermal advection-diffusion equation to simulate erosion of rock at the Earth's surface, the upward transport of the underlying rock toward the surface, and the changes in a 1-D geotherm with time. Below, we provide a working solution to the basic equation for calculating temperature *T* as a function of depth *z* and time *t* (*you don't need to convert the equation to Python, don't worry*). The equation was originally published by Carslaw and Jaeger (1959)

\begin{equation}
  \large
  T(z,t) = G(z + v_{z} t) + \frac{G}{2} \left[ (z - v_{z} t) \mathrm{e}^{-v_{z} z / \kappa} \mathrm{erfc} \left( \frac{z - v_{z}t}{2 \sqrt{\kappa t}} \right) - (z + v_{z} t) \mathrm{erfc} \left( \frac{z + v_{z}t}{2 \sqrt{\kappa t}} \right) \right]
\end{equation}
*Equation 1. 1D time-dependent heat advection-diffusion equation.*

where $G$ is the initial temperature gradient (increase in temperature with depth), $v_{z}$ is the vertical advection velocity (positive upward), $\kappa$ is the thermal diffusivity and $\mathrm{erfc}()$ is the complementary error function, defined as

\begin{equation}
  \large
  \mathrm{erfc}(x) = \frac{2}{\sqrt{\pi}} \int_{x}^{\infty} \mathrm{e}^{-u^{2}} du
\end{equation}
*Equation 2. The complementary error function.*

With this equation, the temperature increases linearly with depth at the start of the calculation (i.e., $T(z,t=0) = Gz$) and the geotherm will evolve over time as a function of the advection velocity and thermal diffusivity.

If you are curious about how the the advection and diffusion equations are combined in Equation 1, you can find a solution to the steady-state version of the 1D heat transfer equation in the [notes on solving the advection-diffusion equation](https://introqg.github.io/site/lessons/L4/solving-advection.html) from [Lesson 4](https://introqg.github.io/site/lessons/L4/overview.html).

Let's begin by plotting geotherms to get a sense of how our temperature equation works.

**Notice**: Closely follow the instructions! For example, you should be sure to use **exactly** the same variable names mentioned in the instructions because your answers will be automatically graded, and the tests that grade your answers rely on following the same formatting or variable naming as in the instructions.

**Your score on this problem will be based on following criteria**:

- Modifying the provided Python function to format and add text to the plots
- Plotting your results and adding figure captions
- Including comments that explain what most lines in the code do
- Uploading your notebook to your GitHub repository for this week's exercise

## Part 1: Setting things up (*0 points*)

To start we can simply run the Python cells below to load the modules we will need and define a few functions. The first two should look somewhat familiar, other than needing to load the `erfc` function from the SciPy module.

- Look over the short descriptions and run the **four** Python cells below.

In [ ]:
# Configure Matplotlib plotting
%matplotlib inline

In [ ]:
# Import modules
import numpy as np
import pandas as pd
from scipy.special import erfc

Next you can run the cell below to load a function for solving the 1D advection-diffusion equation (Equation 1 above).

In [ ]:
def temp_1D(z, time, g_initial, vz, kappa):
    """Calculate solution to 1D transient advection-diffusion heat transfer equation.
    
    Keyword arguments:
    z -- array of depths from surface (units: km)
    time -- array of time since model start (units: m)
    g_initial -- initial temperature gradient (units: deg. C / km)
    vz -- vertical advection velocity (units: km / Ma)
    kappa -- thermal diffusivity (units: km^2 / Ma)
    """
    # Calculate T separately for case where t = 0 to avoid divide-by-zero warnings
    if time == 0:
        temperature = g_initial * z
    else:
        temperature = g_initial * (z + vz * time) + (g_initial / 2.0) * ((z - vz * time) *\
                      np.exp(-(vz * z) / kappa) * erfc((z - vz * time) / (2.0 * np.sqrt(kappa *time))) -\
                      (z + vz * time) * erfc((z + vz * time) / (2.0 * np.sqrt(kappa * time))))
    return temperature

Finally, the cell below defines a function that calls the `temp_1D` function and define some thermal model parameters (among other things). Please read over this function and then run the Python cell below.

In [ ]:
def age_predict(temp_gradient=10.0, time_total=50.0, vz=0.5,
                plot_temp_z_hist=False, n_temp_save=1,
                calc_ahe=False, calc_zhe=False, calc_mar=False):
    """Calculate transient 1D thermal solution and predict thermochronometer ages.
    
    Keyword arguments:
    temp_gradient -- initial thermal gradient (units: deg. C / km; default: 10.0)
    time_total -- total thermal model simulation time (units: Ma; default: 50.0)
    vz -- vertical advection velocity (units: km/Ma; default: 0.5)
    plot_temp_z_hist -- save the temperature-depth history of the tracked particle (default: False)
    n_temp_save -- number of temperature profiles to dave (default: 1)
    calc_ahe -- calculate apatite (U-Th)/He age (default: False)
    calc_zhe -- calculate zircon (U-Th)/He age (default: False)
    calc_mar -- calculate muscovite 40Ar/39Ar age (default: False)
    """

    # THERMAL MODEL PARAMETERS
    z_max = 50.0            # Thermal model thickness [km]
    kappa = 32.0            # Thermal diffusivity [km2 / Ma]
    npz = 101               # Number of depth points for temperature calculation
    npt = 401               # Number of times for temperature calculation
    high_temp = 1000.0      # Temperature to assign if tracked particle depth exceeds zmax
    
    # CLOSURE TEMPERATURE PARAMETERS
    # Apatite (U-Th)/He
    size_ahe = 100.0        # Apatite grain size [um]
    ea_ahe = 138.0e3        # Activation energy [J/mol]
    a_ahe = 25.0            # Geometry factor [25 for sphere]
    d0_ahe = 5.0e-3         # Diffusivity at infinite temperature [m2/s]
    tau_ahe = 1.0           # Initial guess for characteristic time

    # Zircon (U-Th)/He
    size_zhe = 100.0        # Zircon grain size [um]
    ea_zhe = 168.0e3        # Activation energy [J/mol]
    a_zhe = 25.0            # Geometry factor [25 for sphere]
    d0_zhe = 4.6e-5         # Diffusivity at infinite temperature [m2/s]
    tau_zhe = 1.0           # Initial guess for characteristic time

    # Muscovite Ar/Ar
    size_mar = 500.0        # Muscovite grain size [um]
    ea_mar = 183.0e3        # Activation energy [J/mol]
    a_mar = 8.7             # Geometry factor [8.7 for planar sheet]
    d0_mar = 3.3e-6         # Diffusivity at infinite temperature [m2/s]
    tau_mar = 1.0           # Initial guess for characteristic time

    # OTHER CONSTANTS
    r = 8.314               # Universal gas constant

    # Set initial thermochronometer ages
    age_ahe = time_total
    age_zhe = time_total
    age_mar = time_total

    # Convert units
    size_ahe = size_ahe / 1.0e6 / 1.0e3                                     # um -> km
    size_zhe = size_zhe / 1.0e6 / 1.0e3                                     # um -> km
    size_mar = size_mar / 1.0e6 / 1.0e3                                     # um -> km
    d0_ahe = d0_ahe * (1 / 1000.0**2.0) * (1.0e6 * 365.25 * 24.0 * 3600.0)  # m2/s -> km2/Ma
    d0_zhe = d0_zhe * (1 / 1000.0**2.0) * (1.0e6 * 365.25 * 24.0 * 3600.0)  # m2/s -> km2/Ma
    d0_mar = d0_mar * (1 / 1000.0**2.0) * (1.0e6 * 365.25 * 24.0 * 3600.0)  # m2/s -> km2/Ma

    # Calculate diffusion parameter D0/a2 for each mineral
    d0a2_ahe = d0_ahe / size_ahe**2.0    # Apatite
    d0a2_zhe = d0_zhe / size_zhe**2.0    # Zircon
    d0a2_mar = d0_mar / size_mar**2.0    # Muscovite

    # Create thermal model arrays
    z = np.linspace(0.0, z_max, npz)                    # Define depth range array
    time = np.linspace(0.0, time_total, npt)            # Define time range array
    time_ma = np.linspace(time_total, 0.0, npt)         # Define time array in Ma (time before present)
    z_hist = np.linspace(time_total*vz, 0.0, npt)       # Define z particle position history
    temp_hist = np.zeros(npt)                           # Define initial temperature history array
    temp = np.zeros(npz)                                # Define initial temperature array

    # Define increment for saving temperatures
    iout = 0

    # Create temperatures DataFrame
    temp_df = pd.DataFrame(data={'Depth (km)':-z})

    # Loop over all times and calculate temperature
    for i in range(npt):
        # Calculate temperature at time[i]
        temp = temp_1D(z, time[i], temp_gradient, vz, kappa)
        
        # Set the temperature history temperature to high_temp if the tracked particle
        # depth is below z_max (where temperature would be undefined)
        if z_hist[i] > max(z):
            temp_hist[i] = high_temp
        # Otherwise, store the current temperature at the depth of the tracked particle
        else:
            temp_hist[i] = temp_1D(z_hist[i], time[i], temp_gradient, vz, kappa)

        # If the current iteration is one of the save increments, save the geotherm
        if i == iout:
            string = str(time_ma[i])+" Ma"
            temp_df[string] = temp
            iout = iout + int(float(len(time)-1)/float(n_temp_save))

    # Set the initial closure temperatures for the previous step to one
    tc_ahe_prev = 1.0
    tc_zhe_prev = 1.0
    tc_mar_prev = 1.0

    # Set the previous temperature to the max value in the current temperature array
    temp_hist_prev = max(temp)

    # Loop over all positions in the temperature history array
    for i in range(len(temp_hist)):
        # Calculate the cooling rate for the first temperature value
        if i == 0:
            dtdt = (temp_hist[i] - temp_hist[i+1]) / (time_ma[i] - time_ma[i+1])
        # Calculate the cooling rate for the last temperature value
        elif i == len(temp_hist)-1:
            dtdt = (temp_hist[i-1] - temp_hist[i]) / (time_ma[i-1] - time_ma[i])
        # Calculate the cooling rate for the the intermediate temperature values
        else:
            dtdt = (temp_hist[i-1] - temp_hist[i+1]) / (time_ma[i-1] - time_ma[i+1])

        # Ensure the cooling rate is at least 1 deg. C per 10 Ma
        dtdt = max(dtdt, 0.1 / (1.0e6 * 365.25 * 24.0 * 3600.0))

        # Calculate various closure temperatures, if requested
        if calc_ahe:
            age_ahe, tc_ahe, tc_ahe_prev = dodson(tau_ahe, temp_hist[i], temp_hist_prev, time_ma[i],
                                                  time_ma[i-1], age_ahe, tc_ahe_prev, ea_ahe, dtdt,
                                                  a_ahe, d0a2_ahe, r)
        if calc_zhe:
            age_zhe, tc_zhe, tc_zhe_prev = dodson(tau_zhe, temp_hist[i], temp_hist_prev, time_ma[i],
                                                  time_ma[i-1], age_zhe, tc_zhe_prev, ea_zhe, dtdt,
                                                  a_zhe, d0a2_zhe, r)
        if calc_mar:
            age_mar, tc_mar, tc_mar_prev = dodson(tau_mar, temp_hist[i], temp_hist_prev, time_ma[i],
                                                  time_ma[i-1], age_mar, tc_mar_prev, ea_mar, dtdt,
                                                  a_mar, d0a2_mar, r)

        # Store previous temperature in thermal history
        temp_hist_prev = temp_hist[i]

    # Save particle depth-temperature history if requested
    if plot_temp_z_hist:
        ex_hist_df = pd.DataFrame(index=temp_hist, data={'Exhumation history':-z_hist})
    else:
        ex_hist_df = None

    # Plot results
    # Make list of column titles, excluding depth
    cols = temp_df.columns.values[1:]
    # Plot starting geotherm
    ax = temp_df.plot(x=cols[0], y='Depth (km)', figsize=(10,7), label=cols[0])
    # Loop through and plot requested geotherms
    for column in cols[1:]:
        temp_df.plot(x=column, y='Depth (km)', ax=ax, label=column)
    # Plot exhumation history, if requested
    if plot_temp_z_hist:
        # Delete "pass" below and add your commands to plot the exhumation history
        pass

    # Write thermochronometer age(s) to screen, if requested
    # Set to ages to None if age prediction is not requested
    if calc_ahe:
        print("Apatite (U-Th)/He age: {0:.2f} Ma".format(age_ahe))
    else:
        age_ahe = None

    if calc_zhe:
        print("Zircon (U-Th)/He age: {0:.2f} Ma".format(age_zhe))
    else:
        age_zhe = None

    if calc_mar:
        print("Muscovite Ar/Ar age: {0:.2f} Ma".format(age_mar))
    else:
        age_mar = None
    
    return temp_df, ex_hist_df, ax, age_ahe, age_zhe, age_mar

## Part 2: Creating a first plot (*3 points*)

You should be able to call the `age_predict` function defined above with no argument to produce a plot like that shown below.

![Time-dependent heat transfer with advection](img/1D_transient_plot1.png)<br/>
*Figure 1. 1D transient thermal solution including advection.*

- Edit the Python cell containing the `age_predict` function to add axis labels and a title to the plot in produces.
- Determine the advection velocity and edit the cell containing the `age_predict` function to also add a text label showing the advection velocity on your the plot using the `ax.text()` function.
- Re-run the Python cell containing the `age_predict` function to update the version in memory.
- Call the age_predict function in the Python cell below with no arguments to produce your first plot.
    - **Note**: Assign the output of the `age_predict` function to the variable `age_output`. As you can see, the function returns a number of values.
- Add a figure caption for the plot in the cell beneath it describing it as if it were in a scientific journal article.

In [ ]:
# Call the age_predict function below
age_output = None

# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE

In [ ]:
# This test should work
print('The last temperature for 50.0 Ma should be 500.00. My last value is {0:.2f}.'.format(age_output[0].iloc[100,1]))


In [ ]:
# This test should also work
print('The temperature history should be None. My temperature history is {0}.'.format(age_output[1]))

## Part 3: Changing the advection velocity (*2 points*)

Now let's see how the thermal solution changes when you change the advection velocity.

- Call the `age_predict` function in the cell below with an advection velocity of 1.0 km/Ma. Look at the definition of `age_predict` to see how to use a different advection velocity.
    - Again, remember to assign the output to the variable `age_output`.
- Add a figure caption for the plot in the cell beneath it describing it as if it were in a scientific journal article.

In [ ]:
# Call the age_predict function below
age_output = None

# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE

In [ ]:
# This test should work
print('The third temperature for 0.0 Ma should be 33.17. My last value is {0:.2f}.'.format(age_output[0].iloc[2,2]))


## Part 3 (continued)

- In the Python cell below, call the `age_predict` function again with an advection velocity of 0.1 km/Ma.
- Again, add a figure caption for the plot in the cell beneath it describing it as if it were in a scientific journal article.

In [ ]:
# Call the age_predict function below
age_output = None

# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE

In [ ]:
# This test should work
print('The second temperature for 0.0 Ma should be 5.74. My last value is {0:.2f}.'.format(age_output[0].iloc[1,2]))


## Part 4: Changing the simulation time (*1 point*)

Next let's see how the thermal solution changes when you change the simulation time.

- Call the `age_predict` function in the cell below with a simulation time of 100 Ma. Again, look at the definition of `age_predict` to see how to use a different simulation time.
    - Again, remember to assign the output to the variable `age_output`.
- Add a figure caption for the plot in the cell beneath it describing it as if it were in a scientific journal article.

In [ ]:
# Call the age_predict function below
age_output = None

# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE

In [ ]:
# This test should work
print('The tenth temperature for 0.0 Ma should be 106.58. My last value is {0:.2f}.'.format(age_output[0].iloc[9,2]))


## Part 4 (continued)

- In the Python cell below, you can explore what happens for even longer simulation times (1000 Ma, 5000 Ma, etc.). There is no need to add a caption for this plot, but there will be a question about it at the end of this problem.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

## Part 5: The initial thermal gradient (*2 points*)

Now let's see how the thermal solution changes when you change the thermal gradient at the start of the calculation.

- Call the `age_predict` function in the cell below with an initial geothermal gradient of 20°C/km. As before, look at the definition of `age_predict` to see how to use a initial geothermal gradient.
    - Again, remember to assign the output to the variable `age_output`.
- Add a figure caption for the plot in the cell beneath it describing it as if it were in a scientific journal article.

In [ ]:
# Call the age_predict function below
age_output = None

# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE

In [ ]:
# This test should work
print('The last temperature for 50.0 Ma should be 1000.00. My last value is {0:.2f}.'.format(age_output[0].iloc[100,1]))


## Part 5 (continued)

- In the Python cell below, call the `age_predict` function with an initial geothermal gradient of 5°C/km.
- Again, add a figure caption for the plot in the cell beneath it describing it as if it were in a scientific journal article.

In [ ]:
# Call the age_predict function below
age_output = None

# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE

In [ ]:
# This test should work
print('The last temperature for 50.0 Ma should be 250.00. My last value is {0:.2f}.'.format(age_output[0].iloc[100,1]))


## Part 6: Additional temperature calculations (*1 point*)

Finally, let's add a few more calculated geotherms to the plot.

- Call the `age_predict` function with the number of temperature calculations to plot increased from 1 to 5.
    - Again, remember to assign the output to the variable `age_output`.
- Add a figure caption for the plot in the cell beneath it describing it as if it were in a scientific journal article.

In [ ]:
# Call the age_predict function below
age_output = None

# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE

In [ ]:
# This test should work
print('The last temperature for 30.0 Ma should be 596.07. My last value is {0:.2f}.'.format(age_output[0].iloc[100,3]))


## Part 7: Questions for this part (*2.5 points*)

1. How does the thermal solution change when you change the advection velocity?
2. What happens to temperatures in the model as the simulation time increases?
3. What do you observe for the temperatures in the model when you change the simulation time? Are there any potential problems with long simulations? Does the model approach a thermal steady state?
4. How does the initial temperature gradient affect the temperatures in the model? Is there any clear relationship between the initial temperature gradient and the maximum temperature in the model at *t* = 0 Ma?
5. In your opinion, is it helpful to see the temperature calculations at different times?

YOUR ANSWER HERE

# Problem 2 - Cooling ages and exhumation rates (*8.5 points*)

One of the main interests for geoscientists using thermochronology is to determine the average exhumation rate of rocks in a study area based on the ages of thermochronometer data at the surface.

## Predicting thermochronometer ages

Thermochronometers record the time since a rock or mineral was at a certain temperature (the closure temperature) in the Earth.
Above, we have calculated temperature solutions assuming 1-D vertical advection of the crust.
Here, we will track a parcel of rock through the temperature field as it is advected toward the surface and use the recorded temperature history to predict thermochronometer ages.
The key to understanding what is done here is to understand that we will be simulating the position of a parcel of rock at depth in the earth, and at each time step the position of the rock parcel will be moved upward according to the length of the time step multiplied by the advection velocity.
In mathematical terms, this relationship is

\begin{equation}
  \large
  z(t) = z(t-1) - v_{z} dt
\end{equation}
*Equation 3. Equation for calculating rock particle depth as a function of time.*

Thus, we will track a parcel of rock from some depth in the model at the start of the temperature calculation to its final location at the surface when the simulation is complete at 0 Ma.
At each depth, the temperature of the parcel of rock will be stored, which will allow the cooling in the temperature history of the rock parcel to be used to predict different thermochronometer ages.
We will consider the apatite (U-Th)/He, zircon (U-Th)/He, and muscovite <sup>40</sup>Ar/<sup>39</sup>Ar thermochronometers that were presented briefly in lecture.

Thermochronometer closure temperatures will be predicted using Dodson's method, which was also discussed briefly in lecture.
According to Dodson's method, the closure temperature *T*<sub>c</sub> of a thermochronometer is

\begin{equation}
  \large
  T_{\mathrm{c}} = \frac{E_{\mathrm{a}}}{R \ln \left( A \tau D_{0} / a^{2} \right)}
\end{equation}
*Equation 4. The effective closure temperature according to Dodson's method.*

where $E_{\mathrm{a}}$ is the activation energy, $R$ is the universal gas constant, $A$ is a geometric factor ($A = 25$ for a sphere, $A = 8.7$ for a planar sheet), $\tau$ is time for the diffusivity to decrease by a factor of 1/e, $D_{0}$ is the diffusivity at infinite temperature and $a$ is the diffusion domain (we'll assume this is the size of the mineral).
The value of $\tau$ can be calculated as a function of the cooling rate $dT/dt$

\begin{equation}
  \large
  \tau = -\frac{R T^{2}}{E_{\mathrm{a}} dT/dt}
\end{equation}
*Equation 5. The characteristic time for a change in diffusivity.*

By simulating cooling of the minerals by iterating over the values of the recorded temperature history from depth to the surface, thermochronometer ages can be predicted for various systems.

**Notice**: Closely follow the instructions! For example, you should be sure to use **exactly** the same variable names mentioned in the instructions because your answers will be automatically graded, and the tests that grade your answers rely on following the same formatting or variable naming as in the instructions.

**Your score on this problem will be based on following criteria**:

- Modifying the provided Python function to format and add text to the plots
- Plotting your results and adding figure captions
- Including comments that explain what most lines in the code do
- Uploading your notebook to your GitHub repository for this week's exercise

## Part 1: Dodson's method (*0 points*)

As mentioned above and in the lecture materials for this week, Dodson's method can be used to calculate closure temperatures for minerals experiencing temperature-dependent diffusion of a daughter isotope. Once a closure temperature is calculated, the model we are using for recording the temperature of a rock particle as it cools can be used to determine the time at which the rock particle cools below the closure temperature (i.e., the thermochronometer age). We'll do just that in this problem, but first we need to define a function for calculating a closure temperature and thermochronometer age using Dodson's method.

- Run the Python cell below, which defines a function to solve Dodson's equation (i.e., Equations 4 and 5 below).

In [ ]:
def dodson(tau, temp_hist, temp_hist_prev, time, time_prev, age, tc_prev, ea, dtdt, a, d0a2, r):
    """Calculate thermochronometer age and closure temperature using Dodson's method.
    
    Keyword arguments:
    tau -- diffusivity characteristic time (units: Ma)
    temp_hist -- current temperature in history (units: deg. C)
    temp_hist_prev -- temperature from last iteration (units: deg. C)
    time -- time until end of simulation (units: Ma)
    time_prev -- previous time until end of simulation (units: Ma)
    age -- previous calculated thermochronometer age (units: Ma)
    tc_prev -- previous calculated closure temperature (units: deg. C)
    ea -- activation energy (units: J / mol)
    dtdt -- cooling rate (units: deg. C / Ma)
    a -- geometric factor (25 for a sphere, 8.7 for a planar sheet)
    d0a2 -- diffusivity at infinite temperature over domain squared (units: 1 / s)
    r -- universal gas constant (units: J / (mol K))
    """
    # Calculate diffusivity characteristic time
    tau = (r * (temp_hist + 273.15)**2.0) / (ea * dtdt)
    # Calculate new closure temperature
    tc = ea / (r * np.log(a * tau * d0a2)) - 273.15
    # Calculate new cooling age if temperature is above tc
    if temp_hist > tc:
        ratio = (tc_prev - temp_hist_prev)/(tc_prev - temp_hist_prev + temp_hist - tc)
        age = time + (time_prev - time) * ratio
    tc_prev = tc
    return age, tc, tc_prev

## Part 2: Calculating a thermochronometer age (*1.5 points*)

We can start exploring Dodson's method by calculating an apatite (U-Th)/He age with the default parameters.

- Look carefully through the Python cell containing the `age_predict` function to find where the closure temperature is calculated.
- Edit that cell to display the predicted apatite (U-Th)/He age and predicted closure temperature on the plot using the `ax.text()` function.
    - I suggest that you use the text plotting function near the bottom of the `age_predict` function where the predicted apatite (U-Th)/He age is printed to the screen (if requested).
- Re-run the Python cell containing the `age_predict` function to store the updated version in memory.
- Call the age_predict function with calculation of the apatite (U-Th)/He age enabled. Look at the definition of `age_predict` to see how to set the flags (`True`/`False` parameters) that allow you to enable the calculation of different thermochronometers.
    - As in Problem 1, remember to assign the output to the variable `age_output`.
- Add a figure caption for the plot in the cell beneath it describing it as if it were in a scientific journal article.

In [ ]:
# Call the age_predict function below
age_output = None

# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE

In [ ]:
# This test should work
print('The apatite (U-Th)/He age should be 9.07 Ma. My age is {0:.2f} Ma.'.format(age_output[3]))


## Part 3: Predicting other thermochronometer ages (*1.5 points*)

Now we can enable prediction of the zircon (U-Th)/He and muscovite $^{40}$Ar/$^{39}$Ar thermochronometer ages as well.

- Return to the Python cell containing the `age_predict` function to where the closure temperatures are calculated.
- Edit that cell to display the predicted zircon (U-Th)/He and muscovite $^{40}$Ar/$^{39}$Ar ages and predicted closure temperatures on the plot using the `ax.text()` function.
    - Again, I suggest that you use the text plotting function near the bottom of the `age_predict` function where the predicted ages are printed to the screen (if requested).
- Re-run the Python cell containing the `age_predict` function to store the updated version in memory.
- Call the age_predict function with calculation of the all three thermochronometer ages enabled. As above, look at the definition of `age_predict` to see how to set the flags (`True`/`False` parameters) that allow you to enable the calculation of different thermochronometers.
    - As in Problem 1, remember to assign the output to the variable `age_output`.
- Add a figure caption for the plot in the cell beneath it describing it as if it were in a scientific journal article.

In [ ]:
# Call the age_predict function below
age_output = None

# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE

In [ ]:
# This test should work
print('The zircon (U-Th)/He age should be 26.84 Ma. My age is {0:.2f} Ma.'.format(age_output[4]))


## Part 4: Visualizing the thermal history (*2 points*)

To understand more about how the thermochronometer ages are predicted, it can he helpful to look at the temperature-depth history of the parcel of rock as it travels from depth to the surface.

- Return to the Python cell containing the `age_predict` function.
- Edit that cell to replace the placeholder text in the if statement in the code that reads
  ```
  if plot_temp_z_hist:
      # Delete "pass" below and add your commands to plot the exhumation history
      pass
  ```
  with the commands to plot the thermal history in the `ex_hist_df` DataFrame.
    - Plot the values from the `ex_hist_df` DataFrame as black circles.
    - Do not remove the `if plot_temp_z_hist:` statement
- Re-run the Python cell containing the `age_predict` function to store the updated version in memory.
- Call the `age_predict` function with the flag (`True`/`False` parameter) enabled for plotting the temperature-depth history of the rock parcel as it travels from depth to the surface in the model. Again, look at the definition of `age_predict` to see how to set this flag.
    - This should now add a set of symbols to the plot that display how the temperature and depth of the rock parcel changes with time.
    - Keep the three flags for thermochronometer age prediction set to `True`.
    - Also, remember to assign the output to the variable `age_output`.
- Add a figure caption for the plot in the cell beneath it describing it as if it were in a scientific journal article.

In [ ]:
# Call the age_predict function below
age_output = None

# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE

In [ ]:
# This test should work
print('The second depth in the thermal history should be -24.94 km. My depth is {0:.2f} km.'.format(age_output[1].values[1][0]))


## Part 5: Faster advection (*1 point*)

Now we can produce a plot like that above with a faster advection velocity.

- Call the `age_predict` function with the same parameter flags enabled as in the previous part of this problem, but also with an advection velocity of 1.0 km/Ma.
    - Have a look at Problem 1, Part 3 for how to change the advection velocity.
    - Again, remember to assign the output to the variable `age_output`.
- Add a figure caption for the plot in the cell beneath it describing it as if it were in a scientific journal article.

In [ ]:
# Call the age_predict function below
age_output = None

# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE

In [ ]:
# This test should work
print('The second depth in the thermal history should be -49.88 km. My depth is {0:.2f} km.'.format(age_output[1].values[1][0]))


## Part 6: Questions for this problem (*2.5 points*)

1. What is the predicted apatite (U-Th)/He age in Part 2? What does that age mean?
2. What is the closure temperature for Part 2?
3. Do all of the predicted thermochronometer ages make sense in Part 3?
4. In Part 4, do you see any problems with the predicted ages considering the predicted thermochronometer ages and closure temperatures?
5. Do the predicted thermochronometer ages in Part 5 make more sense? Why or why not (think about the difference between Parts 4 and 5)?

YOUR ANSWER HERE

## References

Carslaw, H. S., & Jaeger, J. C. (1959). Conduction of heat in solids. Oxford: Clarendon Press.

Dodson, M. H. (1973). Closure temperature in cooling geochronological and petrological systems. Contributions to Mineralogy and Petrology, 40(3), 259–274.